In [1]:
import pickle
import keras
import tensorflow as tf
#from keras import backend as K
import numpy as np
import sys
import os
sys.path.append(os.path.abspath('../'))
import helpers
import helpers.data_generator
from helpers.data_generator import process_data, DataGenerator
from helpers.custom_losses import denorm_loss, hinge_mse_loss
from helpers.custom_losses import percent_correct_sign, baseline_MAE
from models.LSTMConv2D import get_model_lstm_conv2d, get_model_simple_lstm
from models.LSTMConv2D import get_model_linear_systems, get_model_conv2d
#from utils.callbacks import CyclicLR, TensorBoardWrapper
from keras.callbacks import ModelCheckpoint, ReduceLROnPlateau
from time import strftime, localtime
import matplotlib
from matplotlib import pyplot as plt
import copy
%matplotlib inline
from helpers.normalization import normalize, denormalize, renormalize
from tqdm import tqdm
from sklearn import preprocessing
from sklearn import decomposition
#import tkinter as tk
#from tkinter import filedialog
#root = tk.Tk()
#root.withdraw()

Using TensorFlow backend.
/home/wconlin/.conda/envs/tfgpu/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:458: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/home/wconlin/.conda/envs/tfgpu/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:459: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/home/wconlin/.conda/envs/tfgpu/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:460: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/home/wconlin/.conda/envs/tfg

In [25]:
num_components=10
full_pca_fitter={}
delta_pca_fitter={}
kernels = {}

In [26]:
full_data_path = '/scratch/gpfs/jabbate/full_data/train_data_full.pkl'
test_data_path = '/scratch/gpfs/jabbate/full_data/test_data.pkl' 
profiles = ['temp','dens','press_EFIT01','press_EFIT01','q','q_EFIT01','q_EFIT02','ffprime_EFIT01','ffprime_EFIT02','rotation','itemp','jequil']
for profile in tqdm(profiles):
    traindata, valdata, normalization_dict = helpers.data_generator.process_data(full_data_path,
                                                          [profile],
                                                          normalization_method='RobustScaler',
                                                          window_length=1,
                                                          window_overlap=0,
                                                          lookbacks={profile:0},
                                                          lookahead=1,
                                                          sample_step=1,
                                                          uniform_normalization=True,
                                                          train_frac=1,
                                                          val_frac=0,
                                                          nshots=10000,
                                                          verbose=2,
                                                          flattop_only=True,
                                                          randomize=False,
                                                          pruning_functions=['remove_nan',
                                                                             'remove_dudtrip',
                                                                             'remove_I_coil',
                                                                             'remove_non_gas_feedback',
                                                                             'remove_ECH'],
                                                          excluded_shots=['topology_TOP', 
                                                                          'topology_OUT',
                                                                          'topology_MAR',
                                                                          'topology_IN',
                                                                          'topology_DN',
                                                                          'topology_BOT'],
                                                          delta_sigs=[])
    arr = traindata[profile].squeeze()
    arr = arr[:,1,::2] - arr[:,0,::2]
    full_pca_fitter[profile]=decomposition.IncrementalPCA(n_components=num_components).fit(arr)
    kernels[profile] = {}
    kernels[profile]['variance'] = full_pca_fitter[profile].explained_variance_
    for i in range(num_components):
        arr = full_pca_fitter[profile].components_[:i+1,:]
        kernels[profile][i] = np.dot(arr.T,arr)




  0%|          | 0/12 [00:00<?, ?it/s]

Loading
Signals: temp
Number of useable shots:  7451
Number of shots used:  7451
Total number of timesteps:  773682
Shots with Complete NaN: 
529327 samples total
Removing ECH
Removed 197386 samples
331941 samples remaining
Removing timesteps WITHOUT gas feedback
Removed 54465 samples
277476 samples remaining
Removing weird I-coils
Removed 53742 samples
223734 samples remaining
Removing NaN
Removed 0 samples
223734 samples remaining
Removing dudtrip
Removed 6723 samples
217011 samples remaining
217011 samples remaining after pruning
Total number of samples:  217011
Number of training samples:  217011
Number of validation samples:  0





  8%|▊         | 1/12 [02:04<22:47, 124.34s/it]

Loading
Signals: dens
Number of useable shots:  7440
Number of shots used:  7440
Total number of timesteps:  772676
Shots with Complete NaN: 
528578 samples total
Removing ECH
Removed 197076 samples
331502 samples remaining
Removing timesteps WITHOUT gas feedback
Removed 54289 samples
277213 samples remaining
Removing weird I-coils
Removed 53720 samples
223493 samples remaining
Removing NaN
Removed 0 samples
223493 samples remaining
Removing dudtrip
Removed 6723 samples
216770 samples remaining
216770 samples remaining after pruning
Total number of samples:  216770
Number of training samples:  216770
Number of validation samples:  0





 17%|█▋        | 2/12 [03:35<19:02, 114.28s/it]

...




 83%|████████▎ | 10/12 [14:33<02:46, 83.17s/it]

Loading
Signals: itemp
Number of useable shots:  7007
Number of shots used:  7007
Total number of timesteps:  734689
Shots with Complete NaN: 
503701 samples total
Removing ECH
Removed 187408 samples
316293 samples remaining
Removing timesteps WITHOUT gas feedback
Removed 52015 samples
264278 samples remaining
Removing weird I-coils
Removed 52233 samples
212045 samples remaining
Removing NaN
Removed 0 samples
212045 samples remaining
Removing dudtrip
Removed 6586 samples
205459 samples remaining
205459 samples remaining after pruning
Total number of samples:  205459
Number of training samples:  205459
Number of validation samples:  0





 92%|█████████▏| 11/12 [15:57<01:23, 83.45s/it]

Loading
Signals: jequil
Number of useable shots:  5836
Number of shots used:  5836
Total number of timesteps:  616842
Shots with Complete NaN: 
424351 samples total
Removing ECH
Removed 160977 samples
263374 samples remaining
Removing timesteps WITHOUT gas feedback
Removed 44659 samples
218715 samples remaining
Removing weird I-coils
Removed 43146 samples
175569 samples remaining
Removing NaN
Removed 0 samples
175569 samples remaining
Removing dudtrip
Removed 5205 samples
170364 samples remaining
170364 samples remaining after pruning
Total number of samples:  170364
Number of training samples:  170364
Number of validation samples:  0





100%|██████████| 12/12 [17:16<00:00, 86.40s/it]


In [23]:
with open(os.path.expanduser('~/plasma-profile-predictor/pca_delta_fitters.pkl'),'wb+') as f:
    pickle.dump(full_pca_fitter,f)
with open(os.path.expanduser('~/plasma-profile-predictor/pca_delta_kernels.pkl'),'wb+') as f:
    pickle.dump(kernels,f)

In [10]:
full_pca_fitter['temp'].n_components

10

In [12]:
traindata['jequil'].shape

(14617, 1, 65)

In [20]:
traindata['temp'].shape

(32431, 2, 65)